# Progress log & Notes

This .ipynb contains a journal/log of my progress (starting from )

## Progress before 21-10-2024

### General progress
When I started, the hyperbolic embeddings did not use the **scale_fix** (metric tensor factor), nor the **grad_fix** (correct gradient factor), in the formulation if the gradient.

As I experimented with these factors included (the correct gradient), I noticed that the embeddings (relative to embeddings with the wrong gradient), produced worse visualizations at first sight.

For many/all real world datasets, all the embedding points get pushed along the boundary, resulting in hard to interpret visualizations.

However, upon further investigation, such embeddings may make conceptual sense since the actual visible, discernible, parts of the **Poinare Disk**, (the majority of the visible space) corresponds to a tiny fraction of the whole Hyperbolic Space. Most of the space lives along the boundary of the disk, which is also where all the points are pushed towards. 
So despite worse visulizations (at first sight), in actuality these visualizations probably make sense.

### Notable problems
As I delved into this topic, experimented around, researched etc.., I realized a few things that require careful attention. The following points require some extra attention and could be possible research directions.

1. Interpretation of Hyperbolic embeddings:
    - How do we interpret visualizations of points embedded in Hyperbolic Space?
    - What is the ideal kind of data to be embedded in Hyperbolic Space? (-- A tree-like dataset)
    - How would an ideal embedding of an ideal dataset look like? (-- Similar to hyperbolic tiling?)
    - How does an ideal dataset (-- Custom tree-data set) look like practically once embedded? (-- Not as expected?)
    - Distances visually are not very intuitively graspable. Since the distance function is not linear, we can't apply our Euclidean intuition to interpreting the visuals. On-screen/pixel distances correspond to different **actual** distances in Hyperbolic space, and only that distance should be used for interpretations. 

2. Crowding problem & Hyperbolic space:
    - Is the crowding problem still "present" in Hyperbolic Space? (-- Intuitively no, since we have more space)
    - Area/Volume/... in Hyperbolic space scales exponentially (area of circle grows exponentially), hence the crowding problem may not be a problem?

3. Effect of cost function on embeddings

# 21-10-2024 Progress

### Main progress points
1. Initial experiments with tree-like dataset:
    - No (obvious?) hierarchical/tree-like pattern seems to emerge from the embeddings. (-- Unless I interpret things wrong)

2. Attempt to change correct gradient factor $d^{H}_{ij}$ to $1 + d^{H}_{ij}$ in the hopes of faster convergence
    - Spoiler: It did not help much with our tree-like dataset(s)

3. Changing cost function. Specifically I suspect $q^{H}_{ij}$ to be negatively impacting embedding quality. Since this term uses a t-distribution (heavy tails), it's causing repulsive forces to be propagated too far.
\
Perhaps using a Gaussian helps here?

### Notes
1. I suspected that, Given an affinity matrix where the $p_{ij} > 0 $ between neighbouring (i, j) and $p_{ij} = 0$ for non-neighbouring (i, j), the resulting embedding would lead to $q^{H}_{ij}$'s of similar value, which will naturally reveal a tree-like structure embedding. 
\
This does not seem to be the case however, points are "randomly" scattered around the origin.
\
It also takes many iterations (> 20k) to converge to anything meaningful.

2. It seems natural to use a gaussian for $q^{H}_{ij}$. I will be trying this. (For the reasons already mentioned)
\
I've tried deriving the gaussian gradient, and many things remain similar to the t-sne gradient, however I noticed that in the 1st term things cancel out and you're left with something that does not depend on $p_ij$. Maybe something went wrong here. I'll double check tomorrow.
\
This is also a consequence of taking the $log$ of a gaussian function $exp(..)$ which cancels the exp, thus makes it not depend on $p_ij$ anymore.

### To-do Tomorrow:
1. Double check/derive gaussian gradient. Maybe look at sne paper
2. Implement gaussian gradient & experiment



# 22-10-2024 Progress

### Notes
1. One thing that crossed my mind is whether using some Hyperbolic distribution makes sense.
What even is a Hyperbolic probability distribution? If we define a distribution in some non-Euclidean space, we need to ensure that the distributon still integrates to 1. (Otherwise it's not a valid distribution).
\
In the case of Hyperbolic Space, this (might) mean that we can't just simply replace the distance term $(p_i - p_j)^2$ term with a hyperbolic variant. 
\
However, in our use case, we use softmax probabilities to normalize neighbour probability values so that we do get a proper probability distribution in the end. \
We simply use a probabilistic way (using regular gaussians but with hyperbolic distance) to characterize "neighbourness", and then normalize the values to obtain a valid distribution.
\
**Q**: Do we however, need to think about the distance function used? 

2. Assuming we can simply substitue the squared euclidan distance (in a regular gaussian), with its hyperbolic equivalent. 
- **Regular Gaussian:** $\exp(\frac{-||x_i - x_j||^2}{2 \sigma^2})$ = $\exp(\frac{-r_{ij}}{2 \sigma^2})$ meaning we have: $r_{ij} = ||x_i - x_j||^2$ 
- **Hyperbolic Version:** $\exp(\frac{-r^{H}_{ij}}{2 \sigma^2})$ with $r^{H}_{ij}$ being the Hyperbolic (Poincare disk mdoel) (squared) distance function

We may intuitively (hand-wavily) justify this by assuming that more space in Hyperbolic Space, means that we can just swap back to using a Gaussian over a t-distribution. \
Furthermore, since were in Hyperbolic space, we just swap the Euclidean distance for the Hyperbolic one (Poincare disk model distance). Due to the normalization through softmax probabilities, we still obtain sensible probabilities.

### Gradient (given above assumptions)
This leads us to the following gradient (see paper notes for derivation): \
$\frac{\delta C^H}{\delta y_i} = -2 \sigma^2 * \sum_{j} ((p_{ij} - q^H_{ij}) * d^H_{ij} * \frac{\delta d^H_{ij}}{\delta y_i})$ 

This formula has the following explicit assumptions:
1. $q^H_{ij}$ where we substitute the Euclidean distance for the Hyperbolic one is sensible.
2. $q^H_{ij}$ is a good probabilistic model for neighbourhood-ness. We do not use any explicit symmetric gaussian or whatever (an example is presented in the t-sne paper)

This formula also gives us control over the "thightness" of embedding clusters through the variance term $\sigma^2$ \
We can therefore have much more strong clusters (less spread) in embedding space if we desire that.

**Mathematical justification:** The reasoning for this gradient is mainly heuristic and conceptual. It has no proper mathematical justification behind it. For example, can we just simply replace the Euclidean distance for the Hyperbolic distance? Does this result in any conceptual quirks, differences, etc..? 
These can probably only be answered using mathematical reasoning.


# 23-10-2024 Progress

### Notes
Today I've implemented the gaussian gradient (described above), and everything seems to be working good. However this gradient introduces some problems to:
1. Very small gradients. Compared to t-distribution version, they're actually even smaller.
    - Early exaggeration?
    - Manually scale gradients for x iterations?
    - Initialize embeddings more spread out?
    - Optimize optimization procedure?

2. Difficulty in determining a good $\sigma^2$ (variance) value for $q^H_{ij}$. 
    - $p_ij$ or $V$ is computed using gaussian distances with a pre-defined variance too. Perhaps I can use this variance?
    - $\sigma^2$ can probably be used to control how "spread" out our embeddings will be? Smaller $\sigma^2$ will probably lead to embeddings close to the origin of the disk

3. $p_{ij} - q^H_{ij}$ term may be contributing to small gradients.